In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, Flatten, Dropout, LeakyReLU, Dense
from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose, BatchNormalization
from tensorflow.keras.models import Sequential

(train_images, _), (_, _) = mnist.load_data()
train_images = train_images.reshape(-1, 28, 28, 1).astype('float32') / 127.5 - 1.0  
print("Train images shape:", train_images.shape)

def build_generator():
    model = Sequential([
        Dense(7*7*64, input_shape=(100,)), 
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Reshape((7, 7, 64)),
        Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=True),  
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=True, activation='tanh')
    ])
    return model

def build_discriminator():
    model = Sequential([
        Conv2D(32, (5, 5), strides=(2, 2), padding='same', input_shape=(28, 28, 1), use_bias=True),  
        LeakyReLU(alpha=0.2),
        Dropout(0.4),  
        Conv2D(64, (5, 5), strides=(2, 2), padding='same', use_bias=True),
        LeakyReLU(alpha=0.2),
        Dropout(0.4),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    return model

generator = build_generator()
discriminator = build_discriminator()
d_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.4, clipnorm=1.0)
g_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.4, clipnorm=1.0)
loss_fn = tf.keras.losses.BinaryCrossentropy()
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None, 28, 28, 1], dtype=tf.float32),
    tf.TensorSpec(shape=[None, 28, 28, 1], dtype=tf.float32)
])
def train_discriminator(real_images, fake_images):
    with tf.GradientTape() as tape:
        real_predictions = discriminator(real_images, training=True)
        fake_predictions = discriminator(fake_images, training=True)
        real_loss = loss_fn(tf.ones_like(real_predictions), real_predictions)
        fake_loss = loss_fn(tf.zeros_like(fake_predictions), fake_predictions)
        d_loss = 0.5 * (real_loss + fake_loss)
    gradients = tape.gradient(d_loss, discriminator.trainable_variables)
    for grad, var in zip(gradients, discriminator.trainable_variables):
        if grad is None:
            print(f"Warning: None gradient for variable {var.name}")
    d_optimizer.apply_gradients(zip(gradients, discriminator.trainable_variables))
    return d_loss
@tf.function(input_signature=[tf.TensorSpec(shape=(), dtype=tf.int32)])
def train_generator(batch_size):
    noise = tf.random.normal([batch_size, 100])
    with tf.GradientTape() as tape:
        fake_images = generator(noise, training=True)
        fake_predictions = discriminator(fake_images, training=False)
        g_loss = loss_fn(tf.ones_like(fake_predictions), fake_predictions)
    gradients = tape.gradient(g_loss, generator.trainable_variables)
    for grad, var in zip(gradients, generator.trainable_variables):
        if grad is None:
            print(f"Warning: None gradient for variable {var.name}")
    g_optimizer.apply_gradients(zip(gradients, generator.trainable_variables))
    return g_loss

def save_generated_images(epoch, generator, examples=10):
    noise = np.random.normal(0, 1, (examples, 100))
    generated = generator.predict(noise, verbose=0)
    generated = 0.5 * generated + 0.5
    fig, axs = plt.subplots(1, examples, figsize=(10, 1))
    for i in range(examples):
        axs[i].imshow(generated[i, :, :, 0], cmap='gray')
        axs[i].axis('off')
    plt.savefig(f"gan_output_epoch_{epoch}.png")
    plt.close()
epochs = 20
batch_size = 32  
for epoch in range(epochs):
    print(f"Starting Epoch {epoch}")
    for i in range(len(train_images) // batch_size):
        real_images = train_images[np.random.randint(0, train_images.shape[0], batch_size)]
        noise = np.random.normal(0, 1, (batch_size, 100))
        fake_images = generator(noise, training=False)
        d_loss = train_discriminator(real_images, fake_images)
        g_loss = train_generator(batch_size)

        if i % 50 == 0:  
            print(f"Batch {i}, D Loss: {d_loss.numpy()}, G Loss: {g_loss.numpy()}")

    else: 
        print(f"Epoch {epoch}, D Loss: {d_loss.numpy()}, G Loss: {g_loss.numpy()}")
        save_generated_images(epoch, generator)

TensorFlow Version: 2.19.0
Train images shape: (60000, 28, 28, 1)
Starting Epoch 0
Batch 0, D Loss: 0.7201184034347534, G Loss: 0.7203408479690552
Batch 50, D Loss: 0.38894522190093994, G Loss: 0.3871029019355774
Batch 100, D Loss: 0.3473746180534363, G Loss: 0.3934482932090759
Batch 150, D Loss: 0.2261328101158142, G Loss: 0.665773332118988
Batch 200, D Loss: 0.10789243131875992, G Loss: 1.1203272342681885
Batch 250, D Loss: 0.11313021183013916, G Loss: 0.7053894996643066
Batch 300, D Loss: 0.21448953449726105, G Loss: 0.7832954525947571
Batch 350, D Loss: 0.4642738401889801, G Loss: 0.9528861045837402
Batch 400, D Loss: 0.469396710395813, G Loss: 1.2348098754882812
Batch 450, D Loss: 0.49176424741744995, G Loss: 1.1701765060424805
Batch 500, D Loss: 0.5506314039230347, G Loss: 1.4121313095092773
Batch 550, D Loss: 0.5133316516876221, G Loss: 0.8789684772491455
Batch 600, D Loss: 0.49538981914520264, G Loss: 0.9567239880561829
Batch 650, D Loss: 0.5319229364395142, G Loss: 0.900677084